In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Load the training data from the train.csv file
train_data = pd.read_csv('train.csv')
X_train = train_data.drop('labels', axis=1)
y_train = train_data['labels']
# Preprocess the training data (e.g., handle missing values, remove non-negatives)
# ...

# Split the data into input features (X) and target variable (y)



# Load the test data from the test.csv file
test_data = pd.read_csv('test.csv')

# Preprocess the test data (e.g., handle missing values, remove non-negatives)
# ...

X_test =  test_data.drop('labels',axis=1)
y_test = test_data['labels']

# Perform feature reduction using SelectKBest with chi-squared test
num_features_to_select = 250  # Select top 100 features
selector = SelectKBest(score_func=chi2, k=num_features_to_select)

# Handle missing values in the training and test data using SimpleImputer
imputer = SimpleImputer(strategy='median')  # You can choose a different strategy if desired

# Fit the imputer on the training data and transform both training and test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
scaler = MinMaxScaler(feature_range=(0, 1))  # Scale the data to the range [0, 1]
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled  = scaler.fit_transform(X_test_imputed)

# Apply feature reduction on the imputed data
X_train_reduced = selector.fit_transform(X_train_scaled, y_train)
# Train the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_reduced, y_train)
X_test_reduced = selector.transform(X_test_imputed)

y_pred = xgb_model.predict(X_test_reduced)



# Create a DataFrame to store the predictions
predictions = pd.DataFrame({'Prediction': y_pred})

# Save the predictions to a CSV file
predictions.to_csv('predictions.csv', index=False)


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from sklearn.feature_selection import SelectKBest, chi2
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
X_train = train_data.drop('labels', axis=1)
y_train = train_data['labels']
X_test = test_data.drop('labels', axis=1)
y_test = test_data['labels']
num_features_to_select =230
selector = SelectKBest(score_func=chi2, k=num_features_to_select)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
X_train_reduced = selector.fit_transform(X_train_scaled, y_train)
X_test_reduced = selector.transform(X_test_scaled)
# Reshape the reduced training and test data for LSTM input
timesteps = 1  # Number of time steps
input_dim = X_train_reduced.shape[1]  # Input dimension
X_train_reshaped = X_train_reduced.reshape(-1, timesteps, input_dim)
X_test_reshaped = X_test_reduced.reshape(-1, timesteps, input_dim)
# Build the modified model with Conv1D and MaxPooling1D
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(timesteps, input_dim), padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))  # Additional dense layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))  # Additional dense layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
model.fit(X_train_reshaped, y_train, epochs=1000, batch_size=100)
predictions = model.predict(X_test_reshaped)

# Apply threshold of 0.5 to convert predictions to 0 or 1
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int)

# Convert the ndarray to a DataFrame
predictions_df = pd.DataFrame(predictions_binary, columns=['Predictions'])

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

Epoch 1/1000
32/32 [==============================] - 1s 2ms/step - loss: 0.5913 - accuracy: 0.7262
Epoch 2/1000
32/32 [==============================] - 0s 2ms/step - loss: 0.5701 - accuracy: 0.7391
Epoch 3/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.5658 - accuracy: 0.7391
Epoch 4/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.5586 - accuracy: 0.7391
Epoch 5/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.5467 - accuracy: 0.7391
Epoch 6/1000
32/32 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.7546
Epoch 7/1000
32/32 [==============================] - 0s 2ms/step - loss: 0.4777 - accuracy: 0.7694
Epoch 8/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.4484 - accuracy: 0.7908
Epoch 9/1000
32/32 [==============================] - 0s 2ms/step - loss: 0.4366 - accuracy: 0.7921
Epoch 10/1000
32/32 [==============================] - 0s 2ms/step - loss: 0.4224 - accuracy: 0.8084